In [1]:
import numpy as np
import tensorflow as tf
import gym
env = gym.make('CartPole-v0')

[2017-06-27 10:28:59,849] Making new env: CartPole-v0


# random action policy

In [2]:
env.reset()
random_episodes = 0
reward_sum = 0
while random_episodes < 10:
    env.render()
    observation, reward, done, _ = env.step(np.random.randint(0, 2))
    reward_sum += reward
    if done:
        random_episodes += 1
        print("Reward for this episode was:", reward_sum)
        reward_sum = 0
        env.reset()

('Reward for this episode was:', 23.0)
('Reward for this episode was:', 11.0)
('Reward for this episode was:', 25.0)
('Reward for this episode was:', 21.0)
('Reward for this episode was:', 35.0)
('Reward for this episode was:', 35.0)
('Reward for this episode was:', 30.0)
('Reward for this episode was:', 16.0)
('Reward for this episode was:', 38.0)
('Reward for this episode was:', 11.0)


# discount reward function

In [3]:
def discount_rewards(r):
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(range(r.size)):
        running_add = running_add * gmma + r[t]
        discounted_r[t] = running_add
    return discounted_r

# simple mlp policy

In [4]:
# hidden node nums
h = 50
batch_size = 25
learning_rate = 1e-1
# observation dimension
d = 4
# gmma = reward/discount
gmma = 0.99

# define network architecture
observations = tf.placeholder(tf.float32, [None, d], name="input_x")
w1 = tf.get_variable("W1", shape=[d, h],
                    initializer=tf.contrib.layers.xavier_initializer())
layer1 = tf.nn.relu(tf.matmul(observations, w1))
w2 = tf.get_variable("W2", shape=[h, 1],
                    initializer=tf.contrib.layers.xavier_initializer())
score = tf.matmul(layer1, w2)
probability = tf.nn.sigmoid(score)

# optimizer
tvars = tf.trainable_variables()

adam = tf.train.AdamOptimizer(learning_rate=learning_rate)
w1grad = tf.placeholder(tf.float32, name="batch_grad1")
w2grad = tf.placeholder(tf.float32, name="batch_grad2")
batch_grad = [w1grad, w2grad]
update_grads = adam.apply_gradients(zip(batch_grad, tvars))

# placeholder and loss
input_y = tf.placeholder(tf.float32, [None, 1], name="input_y")
advantages = tf.placeholder(tf.float32, name="reward_signal")
loglik = tf.log(input_y * (input_y - probability) + \
               (1 - input_y) * (input_y + probability))
loss = -tf.reduce_mean(loglik * advantages)
new_grads = tf.gradients(loss, tvars)

# train

In [6]:
xs, ys, drs = [], [], []
reward_sum = 0
episode_number = 1
total_episodes = 10000

with tf.Session() as sess:
    rendering = False
    init = tf.global_variables_initializer()
    sess.run(init)
    observation = env.reset()
    
    grad_buffer = sess.run(tvars)
    for ix, grad in enumerate(grad_buffer):
        grad_buffer[ix] = grad * 0
        
    while episode_number <= total_episodes:
        if reward_sum/batch_size > 100 or rendering == True:
            env.render()
            rendering = True
        
        x = np.reshape(observation, [1, d])
        
        tfprob = sess.run(probability, feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        # reward sum
        xs.append(x)
        y = 1 - action
        ys.append(y)
        
        observation, reward, done, info = env.step(action)
        reward_sum += reward
        drs.append(reward)
        
        # done = experiment finished
        if done:
            episode_number += 1
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            xs, ys, drs = [], [], []
            
            # discount_reward distribution
            discounted_epr = discount_rewards(epr)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            # gradients
            t_grad = sess.run(new_grads, feed_dict={observations: epx,
                              input_y: epy, advantages: discounted_epr})
            for ix, grad in enumerate(t_grad):
                grad_buffer[ix] += grad
            
            # avg reward
            if episode_number % batch_size == 0:
                sess.run(update_grads, feed_dict={
                    w1grad: grad_buffer[0], w2grad: grad_buffer[1]})
                for ix, grad in enumerate(grad_buffer):
                    grad_buffer[ix] = grad * 0
                print('Average reward for episode %d : %f.' % \
                     (episode_number, reward_sum/batch_size))
                
                if reward_sum/batch_size > 119:
                    print("Task solved in", episode_number, 'episodes!')
                    break
                    
                reward_sum = 0
            observation = env.reset()

Average reward for episode 25 : 22.400000.
Average reward for episode 50 : 31.720000.
Average reward for episode 75 : 43.160000.
Average reward for episode 100 : 55.320000.
Average reward for episode 125 : 76.600000.
Average reward for episode 150 : 86.040000.
Average reward for episode 175 : 130.200000.
('Task solved in', 175, 'episodes!')
